In [2]:
# analysis: how does this user's swaps make money
import math
import pandas as pd

from utils import utils

TICK_BASE = 1.0001

def tick_to_price(tick):
    return TICK_BASE ** tick * 10 ** 12

def price_to_tick(price):
    return math.log(price/(10**12), TICK_BASE)


# new csv resolved USDT related swaps => converted to USDC/WETH trading pair
df = pd.read_csv("../data/user_arb_swap_new.csv")
df["datetime"] = df["timestamp"].apply(utils.utc_timestamp_to_date)
df["tick_price"] = df["tick"].apply(tick_to_price)
df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,tick_price
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00,1471.206304
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00,1470.470921
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00,1469.588948
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00,1469.148159
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00,1469.295074


In [3]:
user_data = df.to_dict("records")

for item in user_data:
    if item["tokenIn"] == "USDC":
        item["trade_type"] = "BUY"
        item["trade_eth"] = int(item["amountOut"]) / (10**18)
        item["trade_price"] = item["amountOutUSD"] / item["trade_eth"]
    if item["tokenIn"] == "WETH":
        item["trade_type"] = "SELL"
        item["trade_eth"] = int(item["amountIn"]) / (10**18)
        item["trade_price"] = item["amountInUSD"] / item["trade_eth"]
    item["price_diff"] = item["trade_price"] - item["tick_price"]

    trade_tick = price_to_tick(item["trade_price"])
    item["price_diff_in_tick"] = round(trade_tick - item["tick"])

user_df = pd.DataFrame(user_data)
user_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,tick_price,trade_type,trade_eth,trade_price,price_diff,price_diff_in_tick
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00,1471.206304,SELL,3.516118,1471.65568,0.449376,3
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00,1470.470921,SELL,4.186892,1470.81103,0.340109,2
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00,1469.588948,SELL,3.559441,1469.84919,0.260242,2
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00,1469.148159,BUY,5.613511,1468.78132,-0.366839,-2
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00,1469.295074,SELL,3.454447,1469.56601,0.270936,2


In [7]:
init_price = 1471.206304
init_eth = 100
init_usd = 100 * init_price

# calculate eth, usd in every step to know least investment, considering about trading fee
user_data = user_df.to_dict("records")

last_price = init_price
last_eth = init_eth
last_usd = init_usd

current_usd = 0
current_eth = 0
for item in user_data:
    item["uniswap_fees"] = item["amountInUSD"] * 0.0005
    item["fees"] = item["amountInUSD"] - item["amountOutUSD"]
    if item["tokenIn"] == "USDC":
        current_usd = last_usd - item["amountInUSD"]
        current_eth = last_eth + int(item["amountOut"]) / (10 ** 18)
    if item["tokenIn"] == "WETH":
        current_usd = last_usd + item["amountOutUSD"]
        current_eth = last_eth - int(item["amountIn"]) / (10 ** 18)

    item["current_usd"] = current_usd
    item["current_eth"] = current_eth

    item["tick_portfolio_profit_usd"] = current_usd + current_eth * item["tick_price"] - init_usd - init_eth * item["tick_price"]
    item["trade_portfolio_profit_usd"] = current_usd + current_eth * item["trade_price"] - init_usd - init_eth * item["trade_price"]
    # profit = item["price"] * current_eth + current_usd - item["price"] * init_eth - init_usd
    # item["profit"] = profit

    last_usd = current_usd
    last_eth = current_eth


user_df = pd.DataFrame(user_data)
user_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,...,trade_eth,trade_price,price_diff,price_diff_in_tick,uniswap_fees,fees,current_usd,current_eth,tick_portfolio_profit_usd,trade_portfolio_profit_usd
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,...,3.516118,1471.65568,0.449376,3,2.587257,3.851007,152291.294039,96.483882,-2.270948,-3.851007
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,...,4.186892,1470.81103,0.340109,2,3.079064,3.726801,158445.694543,92.296990,-1.988059,-4.607919
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,...,3.559441,1469.84919,0.260242,2,2.615921,2.902046,163674.634556,88.737549,2.830066,-0.100902
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,...,5.613511,1468.78132,-0.366839,-2,4.125401,5.781870,155423.832653,94.351060,4.071810,6.144062
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,...,3.454447,1469.56601,0.270936,2,2.538269,2.806839,160497.563341,90.896613,1.370993,-1.095444


In [8]:
columns = ["datetime", "trade_type", "trade_eth", "trade_price", "current_usd", "current_eth", "trade_portfolio_profit_usd", "tick_portfolio_profit_usd", "tick_price", "price_diff", "price_diff_in_tick", "fees", "uniswap_fees", "tick", "amountIn", "amountOut", "amountInUSD", "amountOutUSD"]
user_df[columns].head(20)

,datetime,trade_type,trade_eth,trade_price,current_usd,current_eth,trade_portfolio_profit_usd,tick_portfolio_profit_usd,tick_price,price_diff,price_diff_in_tick,fees,uniswap_fees,tick,amountIn,amountOut,amountInUSD,amountOutUSD
0,2023-02-13 16:37:09+00:00,SELL,3.516118,1471.65568,152291.294039,96.483882,-3.851007,-2.270948,1471.206304,0.449376,3,3.851007,2.587257,-203382,3516117741730628903,5170663639,5174.514646,5170.663639
1,2023-02-13 16:37:16+00:00,SELL,4.186892,1470.81103,158445.694543,92.296990,-4.607919,-1.988059,1470.470921,0.340109,2,3.726801,3.079064,-203387,4186892251423356044,6154400504,6158.127305,6154.400504
2,2023-02-13 16:37:26+00:00,SELL,3.559441,1469.84919,163674.634556,88.737549,-0.100902,2.830066,1469.588948,0.260242,2,2.902046,2.615921,-203393,3559441400205692687,5228940013,5231.842059,5228.940013
3,2023-02-13 16:37:35+00:00,BUY,5.613511,1468.78132,155423.832653,94.351060,6.144062,4.071810,1469.148159,-0.366839,-2,5.781870,4.125401,-203396,8250801903,5613510956742205013,8250.801903,8245.020033
4,2023-02-13 16:38:02+00:00,SELL,3.454447,1469.56601,160497.563341,90.896613,-1.095444,1.370993,1469.295074,0.270936,2,2.806839,2.538269,-203395,3454446749687345942,5073730688,5076.537527,5073.730688
5,2023-02-13 16:38:06+00:00,SELL,4.179520,1469.34290,166635.247436,86.717093,-2.528011,0.058715,1469.148159,0.194741,1,3.463624,3.070574,-203396,4179519783531603929,6137684095,6141.147719,6137.684095
6,2023-02-13 16:38:09+00:00,SELL,6.522324,1468.75581,176209.200205,80.194769,-0.478667,6.294816,1468.413805,0.342005,2,5.748918,4.789851,-203401,6522324284006071922,9573952769,9579.701687,9573.952769
7,2023-02-13 16:38:12+00:00,SELL,6.110772,1466.06601,185166.445632,74.083996,51.243055,43.635232,1466.359567,-0.293557,-2,1.550389,4.479398,-203415,6110772471946200474,8957245427,8958.795816,8957.245427
8,2023-02-13 16:38:15+00:00,SELL,6.111978,1465.64210,194119.692387,67.972019,57.503933,67.386945,1465.333526,0.308574,2,4.725175,4.478986,-203422,6111977767070282417,8953246755,8957.971930,8953.246755
9,2023-02-13 16:38:20+00:00,BUY,3.236609,1464.72795,189376.021553,71.208628,83.863861,79.082664,1464.894014,-0.166064,-1,2.918451,2.371835,-203425,4743670834,3236609489830158433,4743.670834,4740.752383


In [18]:
current_price = 1468.75581
print(176209.200205 + 80.194769 * current_price - 100 * 1471.206304 - 100 * current_price)

-0.4782946420891676


In [19]:
current_price = 1466.06601
print(185166.445632 + 74.083996 * current_price - 100 * 1471.206304 - 100 * current_price)

51.24265257595107


In [11]:
data = user_df[["trade_type", "trade_eth", "trade_price", "fees"]].to_dict("records")[:8]
len(data)

8

In [14]:
sell_usd = 0
sell_amount = 0
for i, item in enumerate(data):
    if item["trade_type"] == "SELL":
        sell_usd += item["trade_eth"] * item["trade_price"]
        sell_amount += item["trade_eth"]
        print(i+1, sell_usd/sell_amount)

1 1471.6556799999998
2 1471.1965791364014
3 1470.7707435428183
5 1470.4879605928531
6 1470.2346954809273
7 1469.8552207372281
8 1469.1208293297927


In [ ]:
# q1: 为什么以 trade_price 计价，fees 与 trade_portfolio_profit_usd 相等